# Deployment on the Pynq Z2 SoC

Finally, we will deploy the two models and measure the inference latency on a Pynq Z2 System-on-Chip. You can buy this board yourself for under 200 euros!

<img src="images/pynq.png" alt="pynq" width="500" img align="center"/>

The first thing you have to do, is connect the board following [these instructions](https://pynq.readthedocs.io/en/latest/getting_started/pynq_z2_setup.html). I always connect it to a power source and then directly to my router, connect my computer to the WiFi on the same router, check which IP the pynq gets, then connect to the board in my browser with [http://<board IP address>](http://<board IP address>). You can also connect the board directly to your computer, but that honestly never worked for me on Mac. But please try by following the instructions on the link above! You will be prompted for a password, which is *xilinx*.

You're in!

Now, we need to copy over a few things by pressing the `Upload` button and find:
- The two tarballs we made in the previous exercise "pynq_pack_baseline.tar.gz" and "pynq_pack_qkeras.tar.gz"
- This part3_pynqz2.ipynb notebook

That's it!

Let's load our model onto the FPGA and check the inference latency!

In [ ]:
import numpy as np
import os
import tarfile
import shutil

In [ ]:
# Extract the bitfile, driver and some test data
tar = tarfile.open("pynq_pack_baseline.tar.gz")
tar.extractall()
shutil.move("package","pynq_pack_baseline")
driver = [f for f in os.listdir('./pynq_pack_baseline/') if 'driver' in f][0]
shutil.copy(f'pynq_pack_baseline/{driver}', 'pynq_pack_baseline/hls4mlruntime.py')

With the hls4ml driver, it is super easy loading the network onto the Zynq Z2! Let's load it over and do some inferences with the test signal file we also copied over! We can also check the inference latency:

In [ ]:
os.chdir('pynq_pack_baseline')

from hls4mlruntime import NeuralNetworkOverlay

bitfile = [f for f in os.listdir() if '.bit' in f][0]

X = np.load('X.npy').astype(np.float32)
y_ref = np.load('y.npy').astype(np.float32)
nn = NeuralNetworkOverlay(bitfile, X.shape, X.shape )
y_hw, _, _ = nn.predict(X, X.shape, profile=True)
np.testing.assert_allclose(y_hw, y_ref)

Cool! The total latency per inference is roughly 100 kHz, with a latency per sample of around 0.5s/55969 ~ 9 microseconds. So we would need 400 of these if we would want to run inference at the full LHC bunch srossing of 40 MHz! However, this includes data transfer and software overhead! So the "real" latency is still the one we saw from the reports (3 micorseconds).

In the L1T, the data comes from optical fibers whearas in the Pynq its stored in memory, so moving data bvack and forth takes a bit of time, also we dont have the same software overhead in the L1-trigger.

Let's verify that the latency is roughly the same for the quantized model also on the board:

In [ ]:
# Extract the bitfile, driver and some test data
os.chdir('/home/xilinx/jupyter_notebooks/')  

tar = tarfile.open("pynq_pack_qkeras.tar.gz")
tar.extractall()
shutil.move("package","pynq_pack_qkeras")
driver = [f for f in os.listdir('./pynq_pack_qkeras/') if 'driver' in f][0]
shutil.copy(f'pynq_pack_qkeras/{driver}', 'pynq_pack_qkeras/hls4mlruntime.py') 

In [ ]:
os.chdir('pynq_pack_qkeras')  

from hls4mlruntime import NeuralNetworkOverlay

bitfile = [f for f in os.listdir() if '.bit' in f][0]

X = np.load('X.npy').astype(np.float32)
y_ref = np.load('y.npy').astype(np.float32)
q_nn = NeuralNetworkOverlay(bitfile, X.shape, X.shape )
y_hw, _, _ = q_nn.predict(X, X.shape, profile=True)
np.testing.assert_allclose(y_hw, y_ref)

For fun, let's also check out the true and reconstructed muon $p_{T}$ to convince ourselves that the model is on the board and doing... something? Note that we do not expected it to do something amazing, considering the tail we saw before when attemting to reconstruct the SM test data, and this is BSM data which the algorithm has never seen before and where we would like it to "break down"

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(1,2,figsize=(16,8))
fig.suptitle('Real data versus reconsructed data')
axs[0].hist(X[:,],bins=100,label=r'Real data',histtype='step', linewidth=2, facecolor='none', edgecolor='green',fill=True,density=True)
axs[0].hist(y_hw[:,3],bins=100,label=r'Predicted',histtype='step', linewidth=2, facecolor='none', edgecolor='orchid',fill=True,density=True)
# axs[0].semilogy()
axs[0].set(xlabel=u'Leading e/$\gamma$ $p_{T}$ ( Norm. GeV)', ylabel='A.U')
axs[0].legend(loc='best',frameon=False, ncol=1,fontsize='large')

axs[1].hist(X[:,15],bins=100,label=r'Real data',histtype='step', linewidth=2, facecolor='none', edgecolor='green',fill=True,density=True)
axs[1].hist(y_hw[:,15],bins=100,label=r'Predicted',histtype='step', linewidth=2, facecolor='none', edgecolor='orchid',fill=True,density=True)
axs[1].set(xlabel=u'Leading muon $p_{T}$ (Norm. GeV)', ylabel='A.U')
# axs[1].semilogy()
axs[1].legend(loc='best',frameon=False, ncol=1,fontsize='large')

This is definitely not a good generator of BSM signals :)!. But obvisouly, the algorithms that reconstruct BSM data better, are not neccessarily the best anomaly detection algorithms.

You have now seen just how easy it has become to train a quantized and pruned deep neural network, translate it into FPGA firmware and do inference on a real FPGA! Methods like pruning and quantization-aware training has become crucial for edge applications, and deployment of highly parallel implementations of such models onto FPGAs has become significantly easier for non-experts withi librabries like hls4ml.